# Lojistik Regresyon

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection
import random

## Sınıflandırma

Regresyonda sürekli bir değer tahmin etmeye çalışıyorduk - ör. sınav puanı.

Sınıflandırmada, belirli bir veri noktası için bir sınıf etiketi öngörüyoruz - ör. sınavı geçmek/kalmak. Bunun gibi iki sınıflı bir problem (ikili sınıflandırma) veya çok sınıflı bir problem olabilir.

Sadece bir sınıf etiketi öngörmüyoruz, aynı zamanda her sınıf için bir olasılık da tahmin ediyoruz.

Ders geçme örneğimiz şu şekilde resmedilebilir

![logistic_regression](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)

Sınıflandırma yapmak için bir **karar** vermemiz gerekiyor. Bu, bir **karar sınırı** tanımlamayı gerektirir. 
1D'de (yani bir özellik ile) bu bir nokta olacak; 
2D'de (iki özellik ile) bu bir çizgi olacaktır; 
üç özelliği olan bir düzlem olacak; vb...

Yukarıdaki durumumuzda `num_hours_studying` $\gt 2.7$ ise, o zaman `geçti` tahmininde bulunacağımızı söyleyebiliriz.

İki değişkenli bir durumda karar sınırımız şöyle görünebilir

![bivariate_logistic_regression](https://i0.wp.com/ucanalytics.com/blogs/wp-content/uploads/2017/09/Scatter-Plot-with-Boundary-Logistic-Regression.jpg?resize=768%2C578 )

## Hipotez

Lojistik regresyon hipotezimiz doğrusal veya doğrusal olmayabilir. Doğrusal regresyondan farklı olan, sınıf olasılıklarının çıktısını almak istememizdir. Hipotezimizi bir olasılık değeri olarak ifade etmenin bir yoluna ihtiyacımız var.

Bunu yapmak için sigmoid fonksiyonunu kullanıyoruz.

$$\sigma(h) = \frac{e^h}{e^h + 1} = \frac{1}{1 + e^{-h}}$$

![sigmoid](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)

Bu fonksiyonun girdisini $(0, 1)$ aralığına sıkıştıracağını ve bize geçerli bir olasılık değeri verdiğini görebiliriz.

Nihai hipotezimizi elde etmek için verilerin doğrusal veya doğrusal olmayan işlevini sigmoid işlevinden geçireceğiz.

$$h_\theta(\mathbf{x}) = P(y=1|\mathbf{x}) = \sigma(\mathbf{a}^\top\mathbf{x})$$

$y=1$'ın ne anlama geldiğine keyfi olarak karar verebiliriz - örnekte diyelim ki sınavı geçti. Daha sonra sınavda başarısız olma olasılığı şu şekilde verilir:

$$P(y=0|\mathbf{x}) = 1 - P(y=1|\mathbf{x}) = 1 - \sigma(\mathbf{a}^\top\mathbf{x})$$

## Kayıp Fonksiyonu

Ortalama kare hatası (mean squared error (MSE)) burada uygun bir kayıp işlevi değildir. Olasılıklarla uğraşıyoruz, bu nedenle doğal seçim negatif log loss'dur.

$$\text{NLL} = -y \log(h_\theta) - (1 - y)\log(1 - h_\theta))$$

$y$ ya $1$ ya da $0$ olacak, bu nedenle bu iki terimden sadece biri geçerli olacaktır.

## Optimizasyon

Burada kapalı bir form kullanamayız, bu yüzden **gradyan iniş** kullanmamız gerekecek.

Doğrusal regresyon hakkında bilgi edindiğimizde, kayıp yüzeyinin dışbükey olduğunu fark ettik - herhangi bir noktadan küresel minimuma nasıl ulaşacağınızı bilirsiniz.

Buradaki kaybımız son derece dışbükey değildir. Bunun yerine, iyi bir yerel minimum bulmaya çalışmak için ilk başlangıç ​​noktasından akıllı adımlar atmak için yinelemeli bir algoritma kullanmamız gerekiyor. Resim şuna benziyor

![gradient_descent](https://cdn-images-1.medium.com/max/1600/1*f9a162GhpMbiTVTAua_lLQ.png)

Ana fikir, belirli bir noktada kayba göre parametrelerin gradyanını alırsanız, kaybı azaltmak için bu parametreleri nasıl değiştireceğinizi size söyleyecek olmasıdır.

![gradient_descent_2](https://cdn-images-1.medium.com/max/800/0*rBQI7uBhBKE8KT-X.png)

Açıkça bu şemada sağdan başlarsak, kaybımızı azaltmak için $w$'ı **azaltmamız** gerekir. Bu gerçekten ana fikir.

Bu algoritma ile parametreleri güncellemek ve iyileştirmek için bir dizi "adım" atıyoruz. Güncellemenin her adımı

$$\theta' = \theta - \beta \frac{\partial \text{NLL}(h_\theta, y)}{\partial \theta}$$

Burada $\beta$ hiperparametresi çok önemlidir - adımlarımızın ne kadar büyük olduğunu kontrol eder. Yukarıdaki şemada, öğrenme oranı bizi "U" genişliğinin yarısı kadar bir adım atmış olsaydı, sadece bir yandan diğer yana zıplardık ve asla optimum seviyeye yerleşmezdik.

Attığımız adım sayısına da dikkat etmemiz gerekiyor. Diyagramda sadece iki tane alsaydık, algoritmaya yerleşmek için zaman vermezdik. Pratikte ikiden çok daha fazla adım atacağız. Bu hiperparametreyi ayarlamanız gerekeceğini unutmayın.

## Veri Kümesi

Güney Afrika'nın Western Cape bölgesinin kalp hastalığı yüksek riskli bölgesindeki erkeklerin retrospektif bir örneği. Koroner kalp hastalığı (KKH) vakası başına kabaca iki kontrol vardır. KKH pozitif erkeklerin çoğu, KKH olaylarından sonra risk faktörlerini azaltmak için kan basıncını düşürme tedavisi ve diğer programlardan geçmiştir. Bazı durumlarda ölçümler bu tedavilerden sonra yapılmıştır. Bu veriler, Rousseauw ve diğerleri, 1983, South African Medical Journal'da açıklanan daha büyük bir veri kümesinden alınmıştır. https://web.stanford.edu/~hastie/ElemStatLearn/ adresinden indirilmiştir.

Özellikler:
- sbp: sistolik kan basıncı
- tütün (tobacco): kümülatif tütün (kg)
- ldl: düşük yoğunluklu lipoprotein kolesterol
- yağlanma (adiposity)
- famhist: ailede kalp hastalığı öyküsü (Var, Yok)
- typea: A tipi davranış
- obezite (obesity)
- alkol (alcohol): mevcut alkol tüketimi
- yaş: başlangıç yaşı
- chd: yanıt, koroner kalp hastalığı

## Veri Önişleme

### Verileri Yükleme

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day11-DeepLearning/notebooks/"
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART2/Day12-Optimization/notebooks/"

DATASET_PATH = ROOT_DIR + "datasets/"

In [ ]:
file_name = DATASET_PATH + 'SAheart.data'
data = pd.read_csv(file_name, sep=',', index_col=0)

In [ ]:
data.head()

### Kategorik Değişkenleri Düzenleme

Bir kez daha, one-hot encoding olarak düzenlenmesi gereken kategorik bir değişkenimiz olan 'famhist' var. One-hot encoding, ikili değişkenler listesindeki ("0" veya "1") bir konuma bir sınıf etiketi atar ve istenen sınıfa karşılık gelen konuma bir "1" koyar.

Böylece "famhist" için iki ikili değişkenin one-hot encoding olarak düzenlenecek.

$$
\begin{align}
\text{famhist: Present} \rightarrow \begin{bmatrix} 1 & 0 \end{bmatrix} \\
\text{famhist: Absent} \rightarrow \begin{bmatrix} 0 & 1 \end{bmatrix} \\
\end{align}
$$

In [ ]:
data['famhist_true'] = data['famhist'] == 'Present'
data['famhist_false'] = data['famhist'] == 'Absent'
data = data.drop(['famhist'], axis=1)

## Veri Görselleştirme

Şimdi verilerdeki ilişkilere bir göz atalım. Sınıflandırma için yine de dağılım grafiklerini kullanabiliriz, ancak biraz farklı görünüyorlar. Yine daha sonra kullanmak için uygun bir fonksiyon yapalım. Göreceğiniz gibi, bu çizimleri biraz daha net hale getirmek için çizim boyutunu genişleteceğiz.

In [ ]:
def plot_feature(data, feature_name):
    plt.figure(figsize=(10, 3))
    plt.scatter(data[feature_name], data['chd'])
    plt.xlabel(feature_name)
    plt.ylabel('chd')
    plt.show()

Birlikte bir özelliğe göz atacağız, ardından veri kümesinin geri kalanını kendi başınıza keşfetmeli ve hangi özelliklerin yararlı olacağı konusunda kendi görüşlerinizi oluşturmalısınız.

In [ ]:
"""
Feature list:
sbp tobacco ldl adiposity famhist_true famhist_false typea obesity alcohol age
"""
plot_feature(data, 'age')

In [ ]:
X = data.loc[:, data.columns != 'chd']
y = data['chd']

In [ ]:
X

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)

indices = np.argsort(importances)

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.barh(range(X.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")
# If you want to define your own labels,
# change indices to a list of labels on the following line.
plt.yticks(range(X.shape[1]), indices)
plt.ylim([-1, X.shape[1]])
plt.show()

In [ ]:
feature_index = 6
X.columns[feature_index]

Özellik seçimini daha sonra yapacağımız için, bu sefer bir veri setini `train` ve `test` olarak ayırmak için uygun bir fonksiyon yapmak için zaman ayıralım ve ayrıca bizim için her iki sette $x$ ve $y$'ı ayıralım. Bunu sizin için yapıyoruz.

In [ ]:
def split(data):
    # control randomization for reproducibility
    np.random.seed(42)
    random.seed(42)
    train, test = model_selection.train_test_split(data)
    x_train = train.loc[:, train.columns != 'chd']
    y_train = train['chd']
    x_test = test.loc[:, test.columns != 'chd']
    y_test = test['chd']
    return x_train, y_train, x_test, y_test

## Temel Model Eğitimi

### Değerlendirme

Sınıflarımızın etiketini tahmin ettiğimiz için (kalp hastalığı olsun veya olmasın), çok daha sezgisel bir performans ölçüsüne sahibiz: tahmin doğruluğu. Bunu hesaplamak için `sklearn.metrics.accuracy_score`u kullanacağız - buraya bakın http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html.

In [ ]:
def evaluate(model, x_train, y_train, x_test, y_test):
    train_preds = model.predict(x_train)
    test_preds = model.predict(x_test)
    train_acc = metrics.accuracy_score(y_train, train_preds)
    test_acc = metrics.accuracy_score(y_test, test_preds)
    print('Train accuracy: %s' % train_acc)
    print('Test accuracy: %s' % test_acc)

### Gradyan İniş Modeli

Burada bir temel gradyan iniş modeli uygulanacak. Özellik seçimi veya ayar düzenlemesi yapılacak. 

Temel olarak, `sklearn.linear_model.SGDClassifier` sınıfını kullanır - burada http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier'a başvurun. `loss='log'` argümanını ileterek bir lojistik regresyon modeli elde ederiz.

Temel (baseline) olarak işaretlemek için buradaki değişkenlere `bl` ekleniyor. Ayrıca verileri bölecek, modeli eğitecek ve eğitim ve test doğruluklarını döndürecek bir kullanışlı yardımcı işlev daha ekleyelim.

In [ ]:
def split_train_evaluate(model, data):
    x_train, y_train, x_test, y_test = split(data)
    model.fit(x_train, y_train)
    evaluate(model, x_train, y_train, x_test, y_test)


model_bl = linear_model.SGDClassifier(loss='log', max_iter=10000)
split_train_evaluate(model_bl, data)

### Grid Search

Bu görevle ilgili zor olan şey, eğitimlerimiz sırasında ilk kez bir dizi hiper parametreye sahip olmamızdır. Bu, makine öğrenimi uygulamasındaki yaygın durumdur. Bu sorunla başa çıkmak için bir yaklaşım, her hiperparametre için makul bir değer kümesi tanımlamak ve ardından eğitim kümesinde çapraz doğrulama (cross validation) kullanarak bunların tüm kombinasyonlarını aramaktır. Bu tekniğe **grid search** denir.

`sklearn.model_selection.GridSearchCV` ile Grid Search yapabiliriz, buraya bakın http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html.

Nasıl çalıştığını görelim.

In [ ]:
# this may take a few seconds to run
x_train, y_train, x_test, y_test = split(data)

grid_search = model_selection.GridSearchCV(
    estimator=linear_model.SGDClassifier(loss='log'),
    param_grid={'alpha': [0.01, 0.1, 1.],
                'max_iter': [1000, 10000]},
    cv=10,
    return_train_score=True)


grid_search.fit(x_train, y_train)

Grid Search sonuçları, parametre seçeneklerinin ne yaptığını görebilmeniz için görüntüleme için doğrudan bir `DataFrame` içine yüklenebilen bir dict tipinde gelir.

In [ ]:
r = pd.DataFrame(grid_search.cv_results_)
# we only want a subset of the columns for a precise summary
r[['params', 'mean_train_score', 'mean_test_score']].head()

Grid Search ayrıca sizin için en iyi modeli otomatik olarak seçer ve daha sonra en uygun parametrelerle kullanabilirsiniz.

In [ ]:
best_model = grid_search.best_estimator_
print(grid_search.best_params_)
evaluate(best_model, x_train, y_train, x_test, y_test)

### Örnek - 2

En iyi modelinizi bulmak ve yüzde 72,4'lük temel test doğruluğumuzu geçmek için yukarıdaki Grid Search kodunu kullanabilirsiniz.

'SGDClassifier' belgelerindeki (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) parametreleri incelemek ve kendi kararlarınızı vermek isteyeceksiniz. makul bir arama alanı nasıl görünmelidir.

Ayrıca, bir parametre için en iyi değeriniz arama alanınızın kenarındaysa, tırmanmaya devam edip edemeyeceğinizi görmek için alanı bu yönde daha da genişletmek istediğinizi unutmayın. Örneğin, $\alpha$ için alanım `[0.1, 0.5, 1.]` ise ve en iyi sonuç `1.` ile geldiyse, kesinlikle `2.` ve `5.` ve benzerlerini denemeliyim. .

Ayrıca özellik seçimini de unutmayın. Görselleştirdiğimiz bölüme dönün. İsterseniz, ridge regresyonu gerçekleştirmek için parametreler aracılığıyla bazı $L2$ cezalarını deneyin ve uygulayın.

In [ ]:
def get_feature_set(data, wanted_features):
    return data.loc[:, [col in wanted_features for col in data.columns]]

In [ ]:
param_grid = {
    'alpha': [0.1, 0.3,  1.,  2.,  5.],
    'max_iter': [1000, 10000, 100000]
}

#param_grid = {
#    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
#    'alpha': [0.01, 0.06, 0.1, 0.3,  1.],
#    'max_iter': [1000, 10000, 100000]
#}

Özellik seçimini gerçekleştirmek için `data.columns` öğesini kendi seçtiğiniz özelliklerin bir listesiyle değiştirin - ör. `['feature1', 'feature2']`'.

In [ ]:
#wanted_features = ['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist_true', 'typea',
#                   'famhist_false', 'obesity', 'alcohol', 'age', 'chd']  # must have chd!

wanted_features = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']  # must have chd!

Özellik kümenizi ve Grid Search parametreleri tanımladıktan sonra, en iyi sonucu elde etmek için aşağıdaki hücreyi çalıştırın. 
Sabırlı olun, özellikle Grid Search büyükse, biraz zaman alabilir!

In [ ]:
# this may take a few seconds to load
my_data = get_feature_set(data, wanted_features)  # feature selection

x_train, y_train, x_test, y_test = split(my_data) # splits
grid_search = model_selection.GridSearchCV(       # perform grid search
    estimator=linear_model.SGDClassifier(loss='log'),
    param_grid=param_grid,
    cv=10,
    return_train_score=True)

grid_search.fit(x_train, y_train)
best_model = grid_search.best_estimator_

print(grid_search.best_params_)

evaluate(best_model, x_train, y_train, x_test, y_test)